In [1]:
import pandas as pd
import numpy as np

## Cleaning Census Data

In [2]:
census = pd.read_csv('../data/census_bureau_acs_county_2018_5yr.csv')

In [3]:
census.shape

(3220, 242)

In [4]:
census.isnull().sum().sort_values(ascending=False)[:12]

speak_spanish_at_home_low_english     3220
pop_divorced                          3220
pop_5_years_over                      3220
pop_15_and_over                       3220
pop_never_married                     3220
pop_now_married                       3220
pop_widowed                           3220
pop_separated                         3220
speak_spanish_at_home                 3220
speak_only_english_at_home            3220
aggregate_travel_time_to_work          158
some_college_and_associates_degree      79
dtype: int64

In [5]:
for col in census.columns:
    if census[col].isna().sum() > 100:
        census.drop(columns=col, inplace=True)

**Interpretation:** We dont want any columns with greater than 100 null values

In [6]:
census.isnull().sum().sort_values(ascending=False)[:11]

high_school_including_ged                                      79
less_than_high_school_graduate                                 79
bachelors_degree_2                                             79
some_college_and_associates_degree                             79
different_house_year_ago_same_city                             79
different_house_year_ago_different_city                        79
graduate_professional_degree                                   79
population_1_year_and_over                                     79
not_us_citizen_pop                                             78
owner_occupied_housing_units_lower_value_quartile               5
renter_occupied_housing_units_paying_cash_median_gross_rent     3
dtype: int64

In [7]:
census.dropna(inplace=True)

**Interpretation:** We will use some of these columns in our modeling so we drop the rows instead.

In [8]:
census.shape

(3133, 231)

**Interpretation:** We lost 87 rows or 2.7% of our rows and 11 columns or 4.5% of our columns.

#### Feature Normalization
Many of the columns in this dataset are total numbers. They need to be normalized as percentages of totals before being modeled on.

In [9]:
census['pct_nonfamily_house'] = census['nonfamily_households'] / census['households']
census['pct_family_house'] = census['family_households'] / census['households']
census['pct_male_male_households'] = census['male_male_households'] / census['households']
census['pct_female_female_households'] = census['female_female_households'] / census['households']

census['pct_male'] = census['male_pop'] / census['total_pop']
census['pct_female'] = census['female_pop'] / census['total_pop']
census['pct_white'] = census['white_pop'] / census['total_pop']
census['pct_black'] = census['black_pop'] / census['total_pop']
census['pct_asian'] = census['asian_pop'] / census['total_pop']
census['pct_hispanic'] = census['hispanic_pop'] / census['total_pop']
census['pct_amerindian'] = census['amerindian_pop'] / census['total_pop']
census['pct_other_race'] = census['other_race_pop'] / census['total_pop']
census['pct_two_or_more_races'] = census['two_or_more_races_pop'] / census['total_pop']
census['pct_commuters_by_public_transportation'] = census['commuters_by_public_transportation'] / census['total_pop']

census['pop_per_housing_unit'] = census['total_pop'] / census['housing_units'] 
census['pct_armed_forces'] = census['armed_forces'] / census['total_pop']
census['pct_employed'] = census['employed_pop'] / census['total_pop']
census['pct_bachelors_degree_or_higher_25_64'] = census['bachelors_degree_or_higher_25_64'] / census['total_pop']
census['pct_children'] = census['children'] / census['total_pop']
census['pct_employed_education_health_social'] = census['employed_education_health_social'] / census['total_pop']
census['pct_not_us_citizen_pop'] = census['not_us_citizen_pop'] / census['total_pop']
census['pct_not_hispanic'] = census['not_hispanic_pop'] / census['total_pop']

census['poverty_rate'] = census['poverty'] / census['pop_determined_poverty_status']

## Clean data from healthcare per county

In [10]:
healthcare = pd.read_csv('../data/us_healthcare_capacity-county-CovidCareMap.csv')

In [11]:
healthcare.columns = [col.lower().replace(' ', '_') for col in healthcare.columns]

In [12]:
healthcare.shape

(2537, 22)

In [13]:
healthcare.isnull().sum().sort_values(ascending=False)[:13]

icu_bed_source_last_updated                   2535
icu_bed_occupancy_rate                        1127
all_bed_occupancy_rate                          77
staffed_all_beds_[per_1000_adults_(20+)]        28
licensed_all_beds_[per_1000_elderly_(65+)]      28
licensed_all_beds_[per_1000_adults_(20+)]       28
licensed_all_beds_[per_1000_people]             28
staffed_icu_beds_[per_1000_elderly_(65+)]       28
staffed_icu_beds_[per_1000_adults_(20+)]        28
staffed_icu_beds_[per_1000_people]              28
staffed_all_beds_[per_1000_elderly_(65+)]       28
staffed_all_beds_[per_1000_people]              28
state                                            0
dtype: int64

In [14]:
for col in healthcare.columns:
    if healthcare[col].isna().sum() > 100:
        healthcare.drop(columns=col, inplace=True)

**Interpretation:** We dont want any columns with greater than 100 null values

In [15]:
occupancy_rate_mean = round(healthcare['all_bed_occupancy_rate'].mean(), 2)

In [16]:
healthcare['all_bed_occupancy_rate'].fillna(occupancy_rate_mean, inplace=True)

In [17]:
healthcare.dropna(inplace=True)

**Interpretation:** We will use all bed occupancy rate in our models so we impute the mean. The other rows are dropped.

In [18]:
healthcare['fips_code'][:10]

0    2016
1    2020
2    2050
3    2070
4    2090
5    2110
6    2122
7    2130
8    2150
9    2170
Name: fips_code, dtype: int64

In [19]:
# add leading zeros for 4 digit fips
# healthcare['fips_code'] = healthcare['fips_code'].apply(lambda x: str(int(x)).zfill(5))

**Interpretation:** Without this leading zero many counties would not merge with our other datasets.

## Clean NYT Covid Death data

In [20]:
covid = pd.read_csv('../data/covid19_nyt_us_counties.csv')

In [21]:
covid.shape

(1384683, 6)

In [22]:
covid = covid.groupby('county_fips_code').max()

**Interpretation:** This will give us the most recent covid information for each county which is from 6/3

In [23]:
covid['death_rate'] = covid['deaths']/covid['confirmed_cases']

**Interpretation:** We want to look at deaths rates not total deaths

In [24]:
census.set_index('geo_id', inplace=True)

In [25]:
healthcare.set_index('fips_code', inplace=True)

In [26]:
df = covid.merge(census, left_index = True, right_index=True)

In [27]:
df = df.merge(healthcare, left_index = True, right_index=True)

In [28]:
df.shape, census.shape, healthcare.shape, covid.shape

((2503, 278), (3133, 253), (2509, 19), (3218, 6))

In [29]:
df['date'] = pd.to_datetime(df['date'])
df['do_date'] = pd.to_datetime(df['do_date'])

In [30]:
df.to_csv('../data/cleaned_merged_df.csv')